In [2]:
import requests
import json
import pandas as pd

def send_post_request(url, command, method, selector):
    """
    Function to send a POST request to the Flask API with a specific command and selector.
    
    Args:
    - url (str): The URL of the Flask API endpoint.
    - command (str): The command to be executed on the API.
    - method (str): The method used to find elements (e.g., by class).
    - selector (str): The selector to search for in the document.
    
    Returns:
    - dict: The JSON response from the API as a Python dictionary.
    """
    data = {
        "command": command,
        "params": {
            "method": method,
            "selector": selector
        }
    }

    headers = {
        "Content-Type": "application/json"
    }

    response = requests.post(url, headers=headers, data=json.dumps(data))
    
    if response.status_code == 200:
        print(f"Successfully fetched data for selector '{selector}'.")
        return response.json()
    else:
        print(f"Failed to fetch data. Status Code: {response.status_code}")
        return None


def fetch_product_data(url):
    """
    Function to fetch product data (name, price, price per unit) from the Flask API.
    
    Args:
    - url (str): The URL of the Flask API endpoint.
    
    Returns:
    - dict: A dictionary with lists of product names, prices, and price per unit.
    """
    product_title = send_post_request(url, "find_all", "class", "product__title")
    product_price = send_post_request(url, "find_all", "class", "price")
    product_price_unit = send_post_request(url, "find_all", "class", "priceKil")

    # Check if all data is fetched properly
    if product_title and product_price and product_price_unit:
        return {
            'Product Name': product_title['command_result']['elements'],
            'Price per Unit': product_price_unit['command_result']['elements'],
            'Product Price': product_price['command_result']['elements']
        }
    else:
        print("Error fetching some of the product data.")
        return None


# Function to split price and unit
def split_unit(column):
    
    split_values = column.split(' ')
    # Check if split returns exactly 2 values (price and unit)
    if len(split_values) == 2:
        price, unit = split_values
        return price.replace(",","."),unit
    else:
        # If not, return the original value in both fields
        return None,None
    

def split_price(column):
    
    if column:
        split_values = column.split('\n')
        # Check if split returns exactly 2 values (price and unit)
        if len(split_values) == 2:
            price1, price2 = split_values
            return price1.replace(",","."), price2.replace(",",".")
        else:
            # If not, return the original value in both fields
            return column, column



In [ ]:
# Define the URL of the Flask API endpoint
url = "http://127.0.0.1:5000/execute"
# Fetch product data from the API
product_data = fetch_product_data(url)

In [12]:

def create_dataframe(product_data):
    """
    Function to create a pandas DataFrame from product data.
    
    Args:
    - product_data (dict): A dictionary with lists of product names, prices, and price per unit.
    
    Returns:
    - pd.DataFrame: A pandas DataFrame containing the product data.
    """
    if product_data:
        df = pd.DataFrame(product_data)
        return df
    else:
        print("No data to create DataFrame.")
        return None


df = create_dataframe(product_data)

In [13]:
df['Price per Unit'],df['Measurent Unit per Unit']=zip(*df['Price per Unit'].apply(split_unit))
df[['Price per Unit Initial', 'Price per Unit']] = df['Price per Unit'].apply(split_price).apply(pd.Series)

df['Product Price'],df['Measurent Unit per Product']=zip(*df['Product Price'].apply(split_unit))
df[['Product Price Initial', 'Product Price']] = df['Product Price'].apply(split_price).apply(pd.Series)

df['Unit Price per Difference'] = df['Price per Unit'].astype(float) - df['Price per Unit Initial'].astype(float)

df['Product Price per Difference'] = df['Product Price'].astype(float) - df['Product Price Initial'].astype(float)

df = df[['Product Name', 'Price per Unit',  'Price per Unit Initial','Unit Price per Difference','Measurent Unit per Unit','Product Price', 'Product Price Initial','Product Price per Difference', 'Measurent Unit per Product',]]

In [85]:
import pandas as pd
from datetime import datetime
import os
import win32com.client as win32
import pywintypes

# Helper function to convert pywintypes datetime to naive datetime
def convert_to_naive_datetime(value):
    if isinstance(value, (pywintypes.Time, datetime)):
        pytime = pywintypes.Time(value)
        # Convert to pandas datetime and remove timezone
        return pd.to_datetime(pytime).tz_localize(None)
    return value  # Return the value as-is if it doesn't need conversio

        
                
default_directory = "C:\\Users\\georg\\Python Projects\\Selenium API\\promitheus\Promithies"
pcvs = [7200]


consolidated_df = pd.DataFrame()

# Loop through files in the current directory
for filename in os.listdir(default_directory):
    # Check if the file is an Excel file and starts with one of the specified prefixes
    if filename.endswith(".xls") and any(filename.startswith(str(pcv)) for pcv in pcvs):
        print(filename)
        # Read the Excel file into a DataFrame
        excel = win32.gencache.EnsureDispatch('Excel.Application')
        wb = excel.Workbooks.Open(f"{default_directory}\\{filename}")
        sheet = wb.Sheets(1)  # Access the first sheet
        # Close the workbook and Excel application
        # Extract data to a DataFrame
        data_rows = sheet.UsedRange.Value[1:]
        columns = sheet.UsedRange.Value[0]
        
        data_list = [list(item) for item in data_rows]
        
        # Close the workbook and Excel application
        wb.Close(SaveChanges=False)
        excel.Quit()
        for row in data_list:
            for i in range(4,7):
                date  = datetime.date(pywintypes.Time(row[i]))
                time  = datetime.time(pywintypes.Time(row[i]))
                row[i] = datetime.combine(date, time)
        
        df = pd.DataFrame(data_list, columns=columns)

7200_20241026_100550.xls
7200_20241027_080228.xls
7200_20241027_082547.xls


In [90]:
df["Α/Α Διαγωνιστικής Διαδικασίας"][1]

'359137'

In [87]:
# Convert the prefixes list to strings
pcvs_str = [str(prefix) for prefix in pcvs]

# Define a function to check if any of the codes in the text start with any of the prefixes
def starts_with_prefix(codes):
    # Split the codes by commas
    code_list = codes.split(',')
    # Check if any code starts with any prefix in pcvs_str
    return any(code.startswith(tuple(pcvs_str)) for code in code_list)

# Apply the function to filter the DataFrame
filtered_df = df[df['column_name'].apply(starts_with_prefix)]

0           09331200-0,31158000-8,31720000-9,42967000-2
1                      32441200-8,48000000-8,72000000-5
2     03121100-6,03142500-3,03200000-3,03211000-3,03...
3                      30237200-1,48900000-7,80533100-0
4     30000000-9,30237000-9,30237100-0,30237200-1,48...
5                                            39717200-3
6                                            50720000-8
7                                            31527200-8
8                                            72000000-5
9                                            42997200-3
10                                           31527200-8
11                                30230000-0,30237200-1
12                                           31720000-9
13                                31527200-8,34928500-3
14    30210000-4,30211000-1,30213000-5,30213100-6,30...
15    18521000-7,18523000-1,19520000-7,24500000-9,30...
16                                           72000000-5
17                                           601

In [6]:
from datetime import datetime
import operator
import time
import unittest

import pywintypes

value = data_rows[1][4]


try:
    if type(value) == "<class 'pywintypes.datetime'>":
        pytime_current = pywintypes.Time(value)
        pytime_current = pywintypes.Time(pytime_current)
        print(type(pytime_current))
    else:
        # Try initializing as pywintypes.Time first, if it's a raw date input
        pytime_current = pywintypes.Time(value)
        datetime_obj = datetime.fromisoformat(pytime_current)

   
    
    print("Converted DateTime:", datetime_obj)

except Exception as e:
    print(f"Error converting value '{value}': {e}")

Converted DateTime: 2024-10-03 17:32:17+00:00


In [ ]:
https://nepps-search.eprocurement.gov.gr/actSearch/resources/search/ 359121,1